In [108]:
from faker import Faker
import numpy as np
import pandas as pd
import random
from datetime import timedelta, datetime
import math


In [109]:
#inital distribution (fraudCount alr decreased from transCount and Sum)

preferred_num = {
    "fraudCount": 47225,
    "fraudSum": 2475.0,
    "transCount": 99382070,
    "transSum": 211519515.0,
}

preferred_trans_amount = [
    ((0, 10000), 28333066),
    ((10000, 50000), 33469604),
    ((50000, 100000), 10766020),
    ((100000, 1000000), 21734101),
    ((1000000, 10000000), 4300618),
    ((10000000, 200000000), 825886)
]


#preferred_trans_amount = {
#    "0-10k": 28333066,
#    "10k-50k": 33469604,
#    "50k-100k": 10766020,
#    "100k-1m": 21734101,
#    "1m-10m": 4300618,
#    "10m-": 825886,
#}


generated_num = preferred_num.copy()
generated_trans_amount = preferred_trans_amount.copy()


fake = Faker('hu_HU')


In [110]:
def GenerateAmount():
    range_options, weights = zip(*generated_trans_amount)
    choice = random.choices(
        range_options,
        weights=weights,
        k = 1
    )[0]

    for i, (r, w) in enumerate(generated_trans_amount):
        if r == choice:
            neW = w-1
            generated_trans_amount[i] = (r, neW)

    return round(random.randint(*choice), -1)
    

In [ ]:

start_date = datetime(2024, 10, 1)

data = []

for i in range(1200000):
    sum = GenerateAmount()
    #print(sum)

    fraud = random.choices([0, 1], weights=[generated_num["transCount"], generated_num["fraudCount"]], k = 1)[0]

    if (fraud == 1):
        generated_num["fraudCount"] -= 1
        generated_num["fraudSum"] -= sum
    else:
        generated_num["transCount"] -= 1
        generated_num["transSum"] -= sum

    #kb 13 tranzakcióként telik el egy másodperc ==> egy nap kb 1123200 tranzakció, lefelé 

    row = {
        'id': i + 1,
        'fraud': fraud,
        'name': fake.name(),
        'date': (start_date + timedelta(days=math.floor(i/1123200))).strftime('%Y-%m-%d')
    }
    data.append(row)


df = pd.DataFrame(data)

df.to_csv("./Data/GeneratedDataset.csv", index = False, encoding = 'utf-8')
    